In [208]:
import pandas as pd
import numpy as np
import sqlite3

In [140]:
import requests
from bs4 import BeautifulSoup, SoupStrainer

In [199]:
# Make some room to see stuff (i.e. drop display limits on Pandas rows & cols - be careful w/ big df's!)

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 50)

### The method/code below using BeautifulSoup to access all .csv files in a GitHub Repository is adapted from:

https://stackoverflow.com/questions/69806371/combining-all-csv-files-from-github-repository-link-and-make-it-a-one-csv-file

# 1) State Data

In [141]:
con = sqlite3.connect('test.db')

In [142]:
html = requests.get('https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_daily_reports_us')

for link in BeautifulSoup(html.text, parse_only=SoupStrainer('a')):
    if hasattr(link, 'href') and link['href'].endswith('.csv'):
        url = 'https://github.com'+link['href'].replace('/blob/', '/raw/')
        df = pd.read_csv(url, sep=',', lineterminator='\n')
        
        # Drop rows we're not interested in (e.g. The Cruise Ships with outbreaks, Minor Outlying US Territories)
        df = df[~df['Province_State'].isin(['Diamond Princess', 'Grand Princess', 'American Samoa', 'Recovered', 
                                            'Virgin Islands', 'Guam', 'Northern Mariana Islands'])]
        
        # Account for slight modifications to column nomenclature that occurred over time
        if 'Mortality_Rate' in df.columns:
            df.rename({'Mortality_Rate' : 'Case_Fatality_Ratio', 'People_Tested' : 'Total_Test_Results'}, axis=1, inplace=True)

        # Retain only columns of interest
        df = df.iloc[:, 0:14].drop(columns=['Recovered', 'Active', 'FIPS', 'Total_Test_Results', 'People_Hospitalized'])
        
        # drop data into database
        df.to_sql('state_data', con, if_exists='append', index=False)

In [143]:
con.close()

In [ ]:
'''
SELECT * FROM state_data 
WHERE Province_State IS NULL
OR Last_Update IS NULL
OR Lat IS NULL
OR Long_ IS NULL
OR Confirmed IS NULL
OR Deaths IS NULL
OR Incident_Rate IS NULL
OR Case_Fatality_Ratio IS NULL;
'''

# 2) City Data

In [257]:
largest_200 = pd.read_csv('largest_200_us_cities.csv')
largest_200 = largest_200[['name', 'usps', 'pop2021']]

In [258]:
largest_200.head()

,name,usps,pop2021
0,Anchorage,AK,282958
1,Birmingham,AL,207235
2,Huntsville,AL,205472
3,Montgomery,AL,197777
4,Mobile,AL,186542


In [259]:
# Up to 5 (large states like CA, FL, TX; smaller states may have as few as 0 in the largest 200 - will have to add at least 1 for these)

largest_per_state = largest_200.set_index('name').groupby('usps')['pop2021'].nlargest(5).reset_index()
largest_per_state.head()

,usps,name,pop2021
0,AK,Anchorage,282958
1,AL,Birmingham,207235
2,AL,Huntsville,205472
3,AL,Montgomery,197777
4,AL,Mobile,186542


In [260]:
largest_per_state['usps'].unique().shape

(43,)

In [261]:
state_postal_codes = pd.read_csv('state_postal_codes.csv')[['State', 'Code']]
print(state_postal_codes.shape)
state_postal_codes.head()

(51, 2)


,State,Code
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [262]:
state_postal_codes = pd.concat([state_postal_codes, pd.DataFrame({'State' : ['Puerto Rico'], 'Code' : ['PR']})], axis=0).reset_index(drop=True)
state_postal_codes.shape

(52, 2)

In [263]:
np.setdiff1d(state_postal_codes['Code'], largest_per_state['usps'].unique())

array(['DE', 'ME', 'MT', 'ND', 'NH', 'SC', 'VT', 'WV', 'WY'], dtype=object)

In [264]:
# Will just manually insert the single largest city & population for these states that didn't have a city in the top 200 (above)

small_state_largest = pd.DataFrame({'usps' : np.setdiff1d(state_postal_codes['Code'], largest_per_state['usps'].unique()),
                                    'name' : ['Wilmington', 'Portland', 'Billings', 'Fargo', 'Manchester', 'Charleston', 'Burlington', 'Charleston', 'Cheyenne'],
                                    'pop2021' : [69584, 65835, 109843, 125804, 113035, 140476, 43063, 45264, 65035]})
small_state_largest

,usps,name,pop2021
0,DE,Wilmington,69584
1,ME,Portland,65835
2,MT,Billings,109843
3,ND,Fargo,125804
4,NH,Manchester,113035
5,SC,Charleston,140476
6,VT,Burlington,43063
7,WV,Charleston,45264
8,WY,Cheyenne,65035


In [265]:
largest_per_state = pd.concat([largest_per_state, small_state_largest], axis=0).reset_index(drop=True)
print(largest_per_state.shape)
largest_per_state.head()

(132, 3)


,usps,name,pop2021
0,AK,Anchorage,282958
1,AL,Birmingham,207235
2,AL,Huntsville,205472
3,AL,Montgomery,197777
4,AL,Mobile,186542


In [266]:
# This makes sense: 50 states + DC + Puerto Rico = 52

largest_per_state['usps'].unique().shape

(52,)

In [267]:
largest_per_state = largest_per_state.merge(state_postal_codes, how='left', left_on='usps', right_on='Code').drop(columns=['usps', 'Code'])

In [268]:
print(largest_per_state.shape)
largest_per_state.head()

(132, 3)


,name,pop2021,State
0,Anchorage,282958,Alaska
1,Birmingham,207235,Alabama
2,Huntsville,205472,Alabama
3,Montgomery,197777,Alabama
4,Mobile,186542,Alabama


In [316]:
temp = df[df['Combined_Key'].isin(largest_per_state['Combined_Key'])]
temp = temp[(~temp['Admin2'].isin(['Boise', 'Des Moines', 'Wichita', 'Jackson', 'Lincoln', 'Austin', 'Houston', 'Bayamon', 'San Juan', 'Richmond']))]
temp = temp[['Admin2', 'Province_State']]

In [317]:
temp = temp.rename({'Admin2' : 'name', 'Province_State' : 'State'}, axis=1).reset_index(drop=True)
temp

,name,State
0,Mobile,Alabama
1,Montgomery,Alabama
2,Anchorage,Alaska
3,Fresno,California
4,Los Angeles,California
5,San Diego,California
6,San Francisco,California
7,Denver,Colorado
8,Honolulu,Hawaii
9,Baltimore,Maryland


In [322]:
temp.merge(largest_per_state[['name', 'State', 'pop2021']], how='left', on=['name', 'State']).to_csv('cities_of_interest.csv', index=False)

In [324]:
cities = pd.read_csv('cities_of_interest.csv')
cities

,name,State,pop2021
0,Mobile,Alabama,186542
1,Montgomery,Alabama,197777
2,Anchorage,Alaska,282958
3,Fresno,California,537100
4,Los Angeles,California,3983540
5,San Diego,California,1427720
6,San Francisco,California,883255
7,Denver,Colorado,749103
8,Honolulu,Hawaii,341302
9,Baltimore,Maryland,575584


In [271]:
largest_per_state['Combined_Key'] = largest_per_state['name'] + ', ' + largest_per_state['State'] + ', US'

In [272]:
largest_per_state.head()

,name,pop2021,State,Combined_Key
0,Anchorage,282958,Alaska,"Anchorage, Alaska, US"
1,Birmingham,207235,Alabama,"Birmingham, Alabama, US"
2,Huntsville,205472,Alabama,"Huntsville, Alabama, US"
3,Montgomery,197777,Alabama,"Montgomery, Alabama, US"
4,Mobile,186542,Alabama,"Mobile, Alabama, US"


In [292]:
href = 'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_daily_reports/'

fname = '01-30-2022.csv'

In [293]:
df = pd.read_csv(href+fname+'?raw=true', sep=',', lineterminator='\n')

In [294]:
df.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2022-01-31 04:21:09,33.93911,67.709953,162111,7408,NaN,NaN,Afghanistan,416.434392,4.569708
1,NaN,NaN,NaN,Albania,2022-01-31 04:21:09,41.15330,20.168300,255741,3334,NaN,NaN,Albania,8886.684273,1.303663
2,NaN,NaN,NaN,Algeria,2022-01-31 04:21:09,28.03390,1.659600,250774,6566,NaN,NaN,Algeria,571.876933,2.618294
3,NaN,NaN,NaN,Andorra,2022-01-31 04:21:09,42.50630,1.521800,35556,145,NaN,NaN,Andorra,46018.248884,0.407807
4,NaN,NaN,NaN,Angola,2022-01-31 04:21:09,-11.20270,17.873900,98076,1895,NaN,NaN,Angola,298.409299,1.932175


In [295]:
print(df.shape)
df[df['Combined_Key'].isin(largest_per_state['Combined_Key'])].shape

(4006, 14)


(35, 14)

In [296]:
df[df['Country_Region'] == 'US'].head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
673,1001.0,Autauga,Alabama,US,2022-01-31 04:21:09,32.539527,-86.644082,14498,166,NaN,NaN,"Autauga, Alabama, US",25949.990156,1.144986
674,1003.0,Baldwin,Alabama,US,2022-01-31 04:21:09,30.727750,-87.722071,52459,612,NaN,NaN,"Baldwin, Alabama, US",23499.556519,1.166625
675,1005.0,Barbour,Alabama,US,2022-01-31 04:21:09,31.868263,-85.387129,5241,84,NaN,NaN,"Barbour, Alabama, US",21230.657053,1.602748
676,1007.0,Bibb,Alabama,US,2022-01-31 04:21:09,32.996421,-87.125115,6088,96,NaN,NaN,"Bibb, Alabama, US",27185.853354,1.576873
677,1009.0,Blount,Alabama,US,2022-01-31 04:21:09,33.982109,-86.567906,13981,207,NaN,NaN,"Blount, Alabama, US",24177.705530,1.480581


In [297]:
df[df['Combined_Key'].isin(largest_per_state['Combined_Key'])]

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
721,1097.0,Mobile,Alabama,US,2022-01-31 04:21:09,30.784723,-88.208424,106905,1459,NaN,NaN,"Mobile, Alabama, US",25871.832724,1.364763
723,1101.0,Montgomery,Alabama,US,2022-01-31 04:21:09,32.220683,-86.209693,51697,812,NaN,NaN,"Montgomery, Alabama, US",22825.693420,1.570691
744,2020.0,Anchorage,Alaska,US,2022-01-31 04:21:09,61.149982,-149.142699,91161,468,NaN,NaN,"Anchorage, Alaska, US",31653.125000,0.513377
876,6019.0,Fresno,California,US,2022-01-31 04:21:09,36.757339,-119.646695,220181,2457,NaN,NaN,"Fresno, California, US",22037.912083,1.115900
885,6037.0,Los Angeles,California,US,2022-01-31 04:21:09,34.308284,-118.228241,2648751,28923,NaN,NaN,"Los Angeles, California, US",26384.328805,1.091949
904,6073.0,San Diego,California,US,2022-01-31 04:21:09,33.034846,-116.736533,726071,4643,NaN,NaN,"San Diego, California, US",21749.527458,0.639469
905,6075.0,San Francisco,California,US,2022-01-31 04:21:09,37.752151,-122.438567,119534,712,NaN,NaN,"San Francisco, California, US",13559.541217,0.595646
943,8031.0,Denver,Colorado,US,2022-01-31 04:21:09,39.760183,-104.872569,157363,1176,NaN,NaN,"Denver, Colorado, US",21639.249131,0.747317
1240,15003.0,Honolulu,Hawaii,US,2022-01-31 04:21:09,21.458032,-157.971218,152596,872,NaN,NaN,"Honolulu, Hawaii, US",15657.889741,0.571444
1252,16015.0,Boise,Idaho,US,2022-01-31 04:21:09,43.988299,-115.730928,966,13,NaN,NaN,"Boise, Idaho, US",12335.589324,1.345756


In [279]:
df[df['Combined_Key'].isin(largest_per_state['Combined_Key'])]

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
721,1097.0,Mobile,Alabama,US,2022-01-13 08:00:09,30.784723,-88.208424,90684,1405,NaN,NaN,"Mobile, Alabama, US",21946.225890,1.549336
723,1101.0,Montgomery,Alabama,US,2022-01-13 08:00:09,32.220683,-86.209693,44560,804,NaN,NaN,"Montgomery, Alabama, US",19674.505267,1.804309
744,2020.0,Anchorage,Alaska,US,2022-01-13 08:00:09,61.149982,-149.142699,72782,432,NaN,NaN,"Anchorage, Alaska, US",25271.527778,0.593553
876,6019.0,Fresno,California,US,2022-01-13 08:00:09,36.757339,-119.646695,180217,2404,NaN,NaN,"Fresno, California, US",17713.924818,1.358346
885,6037.0,Los Angeles,California,US,2022-01-13 08:00:09,34.308284,-118.228241,2086581,27850,NaN,NaN,"Los Angeles, California, US",20784.527947,1.334719
904,6073.0,San Diego,California,US,2022-01-13 08:00:09,33.034846,-116.736533,563065,4520,NaN,NaN,"San Diego, California, US",16866.666866,0.802749
905,6075.0,San Francisco,California,US,2022-01-13 08:00:09,37.752151,-122.438567,94278,691,NaN,NaN,"San Francisco, California, US",10001.372584,0.783740
943,8031.0,Denver,Colorado,US,2022-01-13 08:00:09,39.760183,-104.872569,139149,1117,NaN,NaN,"Denver, Colorado, US",19134.611550,0.802737
1240,15003.0,Honolulu,Hawaii,US,2022-01-13 08:00:09,21.458032,-157.971218,109185,815,NaN,NaN,"Honolulu, Hawaii, US",11203.482997,0.746440
1252,16015.0,Boise,Idaho,US,2022-01-13 08:00:09,43.988299,-115.730928,873,11,NaN,NaN,"Boise, Idaho, US",11148.001532,1.260023


In [274]:
df[df['Combined_Key'].isin(largest_per_state['Combined_Key'])]

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
48,2020.0,Anchorage,Alaska,US,2020-04-12 23:18:00,61.149982,-149.142699,127,4,0,123,"Anchorage, Alaska, US"
72,51013.0,Arlington,Virginia,US,2020-04-12 23:18:00,38.876767,-77.101399,366,2,0,364,"Arlington, Virginia, US"
95,48015.0,Austin,Texas,US,2020-04-12 23:18:00,29.885487,-96.277369,10,0,0,10,"Austin, Texas, US"
105,24005.0,Baltimore,Maryland,US,2020-04-12 23:18:00,39.457847,-76.629120,1257,30,0,1227,"Baltimore, Maryland, US"
387,45019.0,Charleston,South Carolina,US,2020-04-12 23:18:00,32.824879,-79.965123,339,2,0,337,"Charleston, South Carolina, US"
413,51550.0,Chesapeake,Virginia,US,2020-04-12 23:18:00,36.676700,-76.305457,121,0,0,121,"Chesapeake, Virginia, US"
616,48113.0,Dallas,Texas,US,2020-04-12 23:18:00,32.766706,-96.777961,525,48,0,1675,"Dallas, Texas, US"
662,8031.0,Denver,Colorado,US,2020-04-12 23:18:00,39.760183,-104.872569,1247,46,0,1201,"Denver, Colorado, US"
663,19057.0,Des Moines,Iowa,US,2020-04-12 23:18:00,40.920126,-91.183266,4,0,0,4,"Des Moines, Iowa, US"
711,37063.0,Durham,North Carolina,US,2020-04-12 23:18:00,36.036385,-78.876083,278,1,0,277,"Durham, North Carolina, US"


In [ ]:
# Cols from 2022 data:

FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio

In [ ]:
# Cols from early 2020 data:

FIPS	Admin2	Province_State	Country_Region	Last_Update	Lat	Long_	Confirmed	Deaths	Recovered	Active	Combined_Key

In [ ]:
# Drop rows we're not interested in (e.g. The Cruise Ships with outbreaks, Minor Outlying US Territories)
df = df[~df['Province_State'].isin(['Diamond Princess', 'Grand Princess', 'American Samoa', 'Recovered', 'Virgin Islands'])]

# Account for slight modifications to column nomenclature that occurred over time
if 'Mortality_Rate' in df.columns:
    df.rename({'Mortality_Rate' : 'Case_Fatality_Ratio', 'People_Tested' : 'Total_Test_Results'}, axis=1, inplace=True)

df = df.iloc[:, 0:14].drop(columns=['Recovered', 'Active', 'FIPS', 'Total_Test_Results', 'People_Hospitalized'])

# Fix problem where Case_Fatality_Ratio was left NULL in some cases before any deaths accrued
# df['Case_Fatality_Ratio'] = df.apply(lambda x: 0 if x['Deaths'] == 0 else x['Case_Fatality_Ratio'], axis=1)

# drop data into database
# df.to_sql('city_data', con, if_exists='append', index=False)